# Решения, использованные при разработке

Мы решили, что не следует умножать сущности без необходимости и отказались от обучения нейросети, доверив всю задачу ChatGPT.

Причины:
1. Не совсем подходящий для обучения датасет.
2. ChatGPT отлично справляется с выделением компаний и текста о ней.
3. ChatGPT хорошо определяет сентимент. При этом можно подправлять его промтом.
4. На выходе мы полули готовое решение для текущей задачи и ПЛЮСОМ хорошо размеченный датасет для возможного обучения других нейросетей.
5. Решение универсальное, его можно использовать и для других подобных задач.

ПЛЮСЫ РЕШЕНИЯ:
- нет необходимости обучать ИИ модели
- получение результата сразу через API
- гибкая настройка промта с параметрами (возможность использовать более точные модели — GPT4)
- простор для увеличения производительности
- легковесность и переносимость

МИНУСЫ:
- подключение через интернет
- требуется vpn/proxy (в рф) для запросов
- оплата за каждое обращение к ChatGPT по API

Пути решения проблем:
- подключение локальных LLM (GigaChat, YandexGPT) вместо ChatGPT
- контурная модель LLM на собственном кластере видеокарт/удаленном сервере на территории РФ

# Сложности

- столкнулись со сложностями, когда ChatGPT не хотела нам отдавать результат в структурированном виде, как нам нужно. Для этого пришлось ее прямо сильно заставить. Использовали инструкцию в промте:
"ВНИМАНИЕ: Оценку следует добавить строго в этой же строке, без переноса на другу строку, даже если строка длинная и в ней есть непонятные символы. Это максимально строгое условие." Если использовать GPT4, будет проще добиться желаемого результата.

- вторая сложность заключалась в том, что в первых проходах модель сообщала, что время обработки займет около трех лет, но нам удалось сократить время до пары минут с помощью методов ускорения, о которых мы расскажем ниже

# Ускорение

## Ускорение 1:
Для того, чтобы ускорить обработку сырых данных, мы распараллелили запросы по ключам API ChatGPT для асинхронной обработки данных.
Это позволило параллельно обрабатывать множество запросов к API, что значительно ускорило обработку данных.

Увеличили скорость в десятки раз.
Всего использовали более 50 ключей.

## Ускорение 2.
Так как ChatGPT определяет компании самостоятельно, нам не нужны дублирующие строки сообщений с различными issurieID. Поэтому мы обработали таблицу, оставив только уникальные сообщения, что позволило избежать лишних запросов.

## Ускорение 3.
При перегрузке ключа API мы его убираем в отдельный список и скрипт не тратит времени на попытку использовать невалидный ключ.

## Ускорение 4.
Подобран ближайшщий к серверам ChatGPT прокси сервер в США, чтобы укоротить маршрут.

## Ускорение 5.
Объединили получение компаний с одновременным определением сентимента в одном промте.
Создали ОДИН промт для приведения сырого сообщения к готовому табличному виду: тикер - описание - оценка сентимента сообщения, отфильтрованные по moex списку. Сократили время вдвое.

## Ускорение 6. На перспективу.
Объединять сообщения в чанки. Чтобы при одном запросе обрабатывалось сразу несколько сырых сообщений. От 3 до 10.